In [1]:
import pandas as pd
import urllib
import os
import string
import re
import numpy as np

In [2]:
df=pd.read_csv("Data/noms_voies_actuelles_paris.csv", sep=';')
df.head()

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,Orig. du nom. ~ Pierre Jean Baptiste Gerbier (...,13.0,174.0,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis."
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"Orig. du nom. ~ M. Joly, propriétaire.",12.0,135.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"Orig. du nom. ~ M. Léchevin, propriétaire.\r\n...",15.0,73.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis."
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"Orig. du nom. ~ François Richard (1765-1839), ...",60.0,1500.0,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla..."
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Orig. du nom. ~ Baruch Spinoza (1632-1677), ph...",12.0,77.0,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81."


In [3]:
len(df)

6475

# 1_Cleaning streets names

On va utiliser la variable "HISTORIQUE", pour extraire des informations relatives à la personne qui a donné son nom à la voie.

In [4]:
# Removing expressions "Origin. du nom"
removed_expressions = ["Orig. du nom. ~ ", "Orig.du nom.~ ", "Orig. Du nom. ~"]
for expression in removed_expressions :
    df["HISTORIQUE"] = df['HISTORIQUE'].astype(str).apply(lambda x: x.replace(expression, ''))

In [5]:
new = df['HISTORIQUE'].str.split("\r\nHistorique. ~ |Historique. ~ ", n = 1, expand = True) 
df['Histoire'] = new[1]
df['Rues'] = new[0]

In [6]:
new = df["Rues"].str.split(";", n = 1, expand = True)
df['Quartier'] = new[1]
df['Rues'] = new[0]

## 1_1 Distinction noms de personnes

On distingue les rues qui portent des noms d'individus et les autres.

Plusieurs façons de reconnaître les rues qui portent des noms d'individus :
- On a souvent les dates de naissance et mort entre parenthèses pour les rues qui portent des noms de personnes
- S'il s'agit d'un propriétaire (qui possédait donc des immeubles dans cette rue), on peut filter avec "propriétaire"
- Pour certaines rues (plutôt des impasses), on n'a pas la description de la personne, car il s'agit d'une rue au voisinage d'un boulevard par exemple (et l'information sur la personne est disponible à la ligne correspondant au boulevard)
- Ou sinon, si on n'a ni l'un ni l'autre, on peut utiliser la liste des prénoms (puisque pour certains individus, on n'a ni les autres informations ci-dessus)

### 1. 1. 1. Voisinages

In [7]:
# dataframe voisinages: on remplace la "Rues" par la vraie référence à la rue
voisinages = df.loc[df['Rues'].str.startswith("Voisinage")].reset_index()
voisinages["Voisinage rue"] = voisinages["Rues"]
voisinages = pd.merge(voisinages.loc[:, voisinages.columns != 'Rues'], 
                      df.loc[~df['Rues'].str.contains("Voisinage"), ['Rues', 'NOM de VOIE']], 
                      how='left', 
                      on='NOM de VOIE')

In [8]:
# Issue: pour certaines rues, on n'a pas exactement le même Libellé voie entre la rue principale et celle au voisinage. 
# Ou sinon, il est juste inscrit "voisinage", même pour la rue principale (si voisinage d'une église par ex)
# On va corriger ça.
list_voisinages_pb = voisinages.loc[voisinages['Rues'].isna(), "NOM de VOIE"].tolist()

for voisinage in list_voisinages_pb :
    if len(df.loc[df['NOM de VOIE'].str.contains(voisinage)]) == len(df.loc[df['NOM de VOIE'].str.startswith(voisinage)]) :
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]
    
    elif len(df.loc[df['NOM de VOIE'].str.contains(voisinage)])==1:
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]

    elif len(df.loc[df['NOM de VOIE'].str.contains(voisinage)])==2:
        new_historique_rue = df.loc[(~df['Rues'].str.contains("Voisinage")) & (df['NOM de VOIE'].str.contains(voisinage)), 'Rues']
        if new_historique_rue.empty :
            voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]
        else :
            voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = new_historique_rue.values[0]
    
    else :
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]

In [9]:
voisinages = voisinages.drop_duplicates(subset='index', keep="first")
voisinages = voisinages.set_index("index")
voisinages.index.name=None

In [10]:
df = df.merge(voisinages.loc[:, ['Rues']], how='left', left_index=True, right_index=True)

In [18]:
df.loc[~df['Rues_y'].isna(), 'Rues_x'] = df['Rues_y']
df = df.drop(['Rues_y'], axis=1).rename(columns={'Rues_x' : 'Rues'})

### 1. 1. 2. Date de naissance et mort, + si c'est un propriétaire

In [23]:
# Dates de naissance et de mort + propriétaire
df['Personne'] = 0
df.loc[df['Rues'].str.contains("\(|propriétaire"), ['Personne']] = 1

In [25]:
df['Personne'].value_counts()

1    3490
0    2985
Name: Personne, dtype: int64

### 1. 1. 3. Avec la liste des prénoms

In [26]:
names = pd.read_csv("Dico_gender_proportion.csv").drop('index', axis=1)
prenoms_list = names['prenoms'].tolist()

In [28]:
df

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier,Personne
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,"Pierre Jean Baptiste Gerbier (1725-1788), avoc...",13.00,174.0,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis.",None,"Pierre Jean Baptiste Gerbier (1725-1788), avocat",quartier où ont été groupés des noms de juris...,1
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"M. Joly, propriétaire.",12.00,135.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse,None,"M. Joly, propriétaire.",None,1
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"M. Léchevin, propriétaire.\r\nHistorique. ~ Av...",15.00,73.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis.",Avant 1942 : passage Léchevin. La partie de ce...,"M. Léchevin, propriétaire.",None,1
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"François Richard (1765-1839), manufacturier d'...",60.00,1500.0,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla...",Cette voie qui recouvre le canal Saint-Martin ...,"François Richard (1765-1839), manufacturier d'...",quartier de fabriques.,1
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Baruch Spinoza (1632-1677), philosophe holland...",12.00,77.0,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81.",None,"Baruch Spinoza (1632-1677), philosophe holland...",None,1
5,NaN,4865,Quinze-Vingts.,rue,NaN,jean bouton,rue Jean Bouton,4775,"M. Jean Bouton, propriétaire.\r\nHistorique. ~...",13.00,98.0,"Classement, alignements (retenus au PAZ de la ...",XIIe,voie publique,"boulevard Diderot, 30, place Henri Frenay et r...","Rue Paul-Henri Grauwin, 22.",Dénommée impasse Jean Bouton avant 1929. Elle ...,"M. Jean Bouton, propriétaire.",None,1
6,NaN,2320,Quinze-Vingts.,avenue,de,corbera,avenue de Corbera,2321,Nom de propriétaire.\r\nHistorique. ~ Précédem...,15.00,102.0,"Classement, alignements (non retenus au POS et...",XIIe,voie publique,"rue de Charenton, 131-135.","rue Crozatier, 11-17.","Précédemment, passage Guillaumot Lainet ; plus...",Nom de propriétaire.,None,1
7,1965-05-05T01:00:00+01:00,6844,Bel Air.,rue,de la,nouvelle-calédonie,rue de la Nouvelle-Calédonie,6787,Ile de la Mélanésie ; territoire d'outre-mer.\...,20.00,120.0,Dénomination. ~ Arr. du 5 mai 1965.,XIIe,voie publique,"boulevard Soult, 20.","rue du Général Archinard, avenue du Général Me...",Ouverte par la Ville de Paris sur l'emplacemen...,Ile de la Mélanésie,territoire d'outre-mer.,0
8,NaN,R530,Picpus.,avenue,des,canadiens,avenue des Canadiens,Z530,Un hôpital militaire accueillant principalemen...,10.00,660.0,Observation. ~ Le premier tronçon est situé da...,XIIe,voie publique,1° tronçon: situé le long de la limite de Pari...,2° tronçon: situé entre la rue du Maréchal Lec...,Le premier tronçon était situé sur les territo...,Un hôpital militaire accueillant principalemen...,None,0
9,1911-03-21T01:00:00+01:00,2904,Picpus.,avenue,NaN,dorian,avenue Dorian,2927,"Pierre Frédéric Dorian (1814-1873), manufactur...",40.00,80.0,Classement. ~ Déc. du 23 septembre 1881.\r\nAl...,XIIe,voie publique,"rue de Picpus, 9.","place de la Nation, 4.",Précédemment rue Dorian (déc. du 25 janvier 18...,"Pierre Frédéric Dorian (1814-1873), manufactur...",None,1


In [29]:
df['Personne2'] = 0
for prenom in prenoms_list :
    df.loc[df['NOM de VOIE'].str.contains(prenom), 'Personne2']=1

In [35]:
df.loc[(df['Personne']==0 & df['Personne2']==1),:]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [40]:
df['HISTORIQUE'][25]

"Historique. ~ voie créée dans le secteur de la Halle Freyssinet, précédemment voie GR/13. Anciennement partie de la voie GM/13.\r\n Grace Murray Hopper (New-York, 1906- comté d'Arlington, 1992), mathématicienne et informaticienne américaine, ayant mis au point le premier compilateur ainsi que le langage Cobol."

In [37]:
df

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,...,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier,Personne,Personne2
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,"Pierre Jean Baptiste Gerbier (1725-1788), avoc...",13.00,...,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis.",None,"Pierre Jean Baptiste Gerbier (1725-1788), avocat",quartier où ont été groupés des noms de juris...,1,0
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"M. Joly, propriétaire.",12.00,...,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse,None,"M. Joly, propriétaire.",None,1,0
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"M. Léchevin, propriétaire.\r\nHistorique. ~ Av...",15.00,...,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis.",Avant 1942 : passage Léchevin. La partie de ce...,"M. Léchevin, propriétaire.",None,1,0
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"François Richard (1765-1839), manufacturier d'...",60.00,...,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla...",Cette voie qui recouvre le canal Saint-Martin ...,"François Richard (1765-1839), manufacturier d'...",quartier de fabriques.,1,1
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Baruch Spinoza (1632-1677), philosophe holland...",12.00,...,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81.",None,"Baruch Spinoza (1632-1677), philosophe holland...",None,1,0
5,NaN,4865,Quinze-Vingts.,rue,NaN,jean bouton,rue Jean Bouton,4775,"M. Jean Bouton, propriétaire.\r\nHistorique. ~...",13.00,...,"Classement, alignements (retenus au PAZ de la ...",XIIe,voie publique,"boulevard Diderot, 30, place Henri Frenay et r...","Rue Paul-Henri Grauwin, 22.",Dénommée impasse Jean Bouton avant 1929. Elle ...,"M. Jean Bouton, propriétaire.",None,1,1
6,NaN,2320,Quinze-Vingts.,avenue,de,corbera,avenue de Corbera,2321,Nom de propriétaire.\r\nHistorique. ~ Précédem...,15.00,...,"Classement, alignements (non retenus au POS et...",XIIe,voie publique,"rue de Charenton, 131-135.","rue Crozatier, 11-17.","Précédemment, passage Guillaumot Lainet ; plus...",Nom de propriétaire.,None,1,0
7,1965-05-05T01:00:00+01:00,6844,Bel Air.,rue,de la,nouvelle-calédonie,rue de la Nouvelle-Calédonie,6787,Ile de la Mélanésie ; territoire d'outre-mer.\...,20.00,...,Dénomination. ~ Arr. du 5 mai 1965.,XIIe,voie publique,"boulevard Soult, 20.","rue du Général Archinard, avenue du Général Me...",Ouverte par la Ville de Paris sur l'emplacemen...,Ile de la Mélanésie,territoire d'outre-mer.,0,1
8,NaN,R530,Picpus.,avenue,des,canadiens,avenue des Canadiens,Z530,Un hôpital militaire accueillant principalemen...,10.00,...,Observation. ~ Le premier tronçon est situé da...,XIIe,voie publique,1° tronçon: situé le long de la limite de Pari...,2° tronçon: situé entre la rue du Maréchal Lec...,Le premier tronçon était situé sur les territo...,Un hôpital militaire accueillant principalemen...,None,0,1
9,1911-03-21T01:00:00+01:00,2904,Picpus.,avenue,NaN,dorian,avenue Dorian,2927,"Pierre Frédéric Dorian (1814-1873), manufactur...",40.00,...,Classement. ~ Déc. du 23 septembre 1881.\r\nAl...,XIIe,voie publique,"rue de Picpus, 9.","place de la Nation, 4.",Précédemment rue Dorian (déc. du 25 janvier 18...,"Pierre Frédéric Dorian (1814-1873), manufactur...",None,1,1


In [32]:
df['Personne'].value_counts()

1    3490
0    2985
Name: Personne, dtype: int64

In [31]:
df['Personne2'].value_counts()

0    3780
1    2695
Name: Personne2, dtype: int64

In [30]:
len(prenoms_list)

2047

# Suite

In [ ]:
df['HISTORIQUE'][2691]
#df['Libellé VOIE'][2691]

In [ ]:
df.loc[df['Rues'].str.contains("fermier"), ['Rues']]

In [ ]:
df.loc[df['Rues'].str.contains("Santos-Dumont")]

In [ ]:
df_autres

In [ ]:
df_autres.loc[(~df['Rues'].str.contains(prenoms_list)), ['Rues', 'Libellé VOIE', 'HISTORIQUE']].dropna(axis=1)

In [ ]:
df['Rues'].str.split(",", n = 1, expand = True)

In [ ]:
df['HISTORIQUE'].str.split(",", n = 1, expand = True) 

In [ ]:
# If "Nom de propriétaire" : chercher dans le nom de la rue, labeliser "homme"
# Problème des noms de profession
# rues qui sont des voisinages
# rues qui contiennent prénom